In [16]:
# Load dataset
import pandas as pd

# load csv
df = pd.read_csv("data/hackupc-travelperk-dataset.csv")
# parse dates
df["Departure Date"] = pd.to_datetime(df["Departure Date"],format="%d/%m/%Y")
df["Return Date"] = pd.to_datetime(df["Return Date"],format="%d/%m/%Y")



# Cities of the trip, date and number of people having the same trip. Same starting and ending city.

In [18]:
from collections import defaultdict

people_by_city = defaultdict(lambda: defaultdict(int))
# iterate on each row of the dataframe
for index, trip in df.iterrows():
    for day in pd.date_range(trip["Departure Date"], trip["Return Date"]):
        # if the day is not in the dictionary, add it
        # print(trip)

        #     if day not in people_by_city[(row["Departure City"],row["Arrival City"])]:
        people_by_city[(trip["Departure City"],trip["Arrival City"])][day] += 1

#  convert defaultdict to dict
people_by_city = {k: dict(v) for k, v in people_by_city.items()}


df2 = pd.DataFrame(
    [
        {"tripcities": cities, "date": date, "num_people": n_people}
        for cities, dates in people_by_city.items()
        for date, n_people in dates.items()
    ]
)

# remove when there is only one person
df2 = df2[df2["num_people"] > 1]
# sort by npersons
df2 = df2.sort_values(by=["num_people"], ascending=False)
df2.to_csv("data/same_trip.csv", index=False)

# Arrival city, day spent in the city, people names and number of people present in the city on that day.

In [5]:
from collections import defaultdict

people_by_city = defaultdict(lambda: defaultdict(list))
# iterate on each row of the dataframe
for index, trip in df.iterrows():
    for day in pd.date_range(trip["Departure Date"], trip["Return Date"]):
        people_by_city[trip["Arrival City"]][day] += [trip["Traveller Name"]]

#  convert defaultdict to dict
people_by_city = {k: dict(v) for k, v in people_by_city.items()}


df2 = pd.DataFrame(
    [
        {"arrival_city": cities, "date": date, "people": n_people}
        for cities, dates in people_by_city.items()
        for date, n_people in dates.items()
    ]
)

df2['people'] = df2['people'].apply(sorted)
df2['num_people'] = df2['people'].apply(len)
df2 = df2[df2["num_people"] > 1]
df2 = df2.sort_values(by=["arrival_city", "date"], ascending=True)
df2.to_csv("data/same_city_people.csv", index=False)